# More Plotly

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
pio.renderers.default='notebook_connected'
pio.templates.default = 'simple_white'

#### First a quick recap...

## Plotting Submodules

Plotly has a couple of different submodules for plotting.  

   -__Plotly Express__ is the high-level interface.
   
   -__Graph Objects__ is the low-level interface.  
    
Express is essentially to Graph Objects what Seaborn is to Matplotlib. Their plotting functions return the same object type....and these objects are customizable. We can generate anything with graph objects that we can generate with express (not the other way around), but it'll require significantly more code. The graph objects submodule requires specifying Figure and Layout and then Adding Traces (plots) to the figure and layout.

Express offers functions to build fully populated Figure objects easily. Figures built with express are still returned as plotly.graph_objects.Figure instances.

See the [api reference](https://plotly.com/python-api-reference/) for additional details.

Additional important submodules are:

   -__Subplots__ contains helper functions for laying out multi-plot figures.
   
   -__Figure Factory__ offers special figures that are difficult to achieve otherwise

## Displaying Figures

There are a number of ways to display Plotly figures.  

- Within the context of a script or notebook we'll use the renderers framework.  
    - Above I've set pio.renderers.default to 'notebook_connected'.  This requires an internet connection.  
    - 'notebook' works without an inernet connection.  
    - 'browser' uses default web browser.  
    - There are webdriver-specific renderers as well.  
    - Static renderers (including png, svg and jpeg)
    - There are several others as well.
    
    
- We can display in a web app using Dash.


- We can export to an HTML file.


- We can also generate static plots using pio.write_image() or the .write_image() graph object method.

In [42]:
gpm = px.data.gapminder()

#### Recall that the px plotting functions support a number of parameters, but the resulting plots can be further tuned with direct access attribute setting or throught the use of update methods.  

#### Here we plot a simple animated scatter plot of lifeExp by gdpPercap with the style specified by passing a template.

In [3]:
fig = px.scatter(gpm, 
                  x='gdpPercap', 
                  y='lifeExp', 
                  color='lifeExp',
                  symbol='continent',
                  size='pop', 
                  hover_data={'country': True,
                              'year': False,
                              'gdpPercap': ':.0f',
                              'lifeExp': ':.0f'
                             }, 
                  labels=dict(lifeExp='Life Expectancy',
                             gdpPercap='GDP Per Capita',
                             pop='population'),
                  animation_frame='year',
                  range_x=[0, 45000],
                  range_y=[0, 85],
                  title='Gapminder: PerCap GDP x Life Exp',
                  template='plotly_dark')

fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0,
    xanchor="right",
    x=1
))

fig.show()

#### Remember, we can inspect the figure tree to see current plot parameters.

In [4]:
print(fig.layout.legend)

layout.Legend({
    'itemsizing': 'constant',
    'title': {'text': 'continent'},
    'tracegroupgap': 0,
    'x': 1,
    'xanchor': 'right',
    'y': 0,
    'yanchor': 'bottom'
})


#### And we can use direct attribute setting as opposed to update methods.

In [5]:
fig.layout.legend.title = ''

fig.show()

Let's circle back to where we left off last week...Here is the in-class assignment we didn't get to...

We've only scratched the surface in terms of available px plotting functions.  Using the gapminder data, build an animated histogram that shows one of the available metrics over the years.  Use the [px.histogram API reference](https://plotly.com/python-api-reference/generated/plotly.express.histogram.html) for details on the available parameters.  

It would be easy to animate over the years, but if you need an extra challenge, consider discretizing one of the continous dimensions and animating over that new variable.

In [43]:
# basic histogram with a marginal rug plot.

fig0 = px.histogram(gpm, 'lifeExp',
                    animation_frame='year',
                    nbins=25,
                    color_discrete_sequence = ['black'] * 12,
                    marginal='rug',
                    range_x=[np.floor(gpm.lifeExp.min()),
                            np.ceil(gpm.lifeExp.max())],
                    title='Global Life Expectancy Distribution 1952 - 2007')

fig0.update_layout(font_family='Times New Roman',
                   title_font_family='Times New Roman',
                   title_font_size=22)
fig0.show()

#### _*Notice that for 1992 we're falling back to the default number of bins._

This is partly because the nbins parameter is actually the maximum number of bins.  The value is used to compute the optimal bin size.  This isn't super clear in the documentation.  Below we examine what the bin width would be for each frame given exactly 25 bins.  Notice the larger range of values for 1992.

In [44]:
avg_s = 0

for i in gpm['year'].unique():
    temp = (gpm.loc[gpm['year']==i, 'lifeExp'].max() - gpm.loc[gpm['year']==i, 'lifeExp'].min()) / 25
    print(i, temp)
    avg_s += temp

print(avg_s/12)

1952 1.75476
1957 1.72552
1962 1.6673200000000004
1967 1.6055999999999997
1972 1.5728
1977 1.7956
1982 1.5466
1987 1.5505600000000002
1992 2.2304399999999998
1997 1.7841199999999997
2002 1.7122799999999996
2007 1.7195999999999998
1.7221


#### While px has an nbins param, its histogram constructor doesn't have a parameter for setting bin width.  This is common when comparing px and go plotting functions. The equivalent graph objects function does have such a parameter, and since px returns a graph objects figure object, we can use update methods to tune our plots.

Below we'll use the **update_traces()** convenience method to specify the xbins parameter.  This will override what we've passed to nbinsx when establishing the bins.

We'll encounter an issue if we just blindly call this method on the figure though, as we have multiple traces due to the rug plot.

The **update_traces()** method supports a selector argument for this exact situation.  This parameter will allow us to specify the traces that should be updated.

Bin size will be about the same as before, but we'll have fixed our bins for 1992.

#### Let's quickly inspect the type of our traces.

In [45]:
print(fig0.data[0].type)
print(fig0.data[1].type)

histogram
box


In [46]:
fig0.update_traces(xbins=dict(start=gpm.lifeExp.min(),
                              end=gpm.lifeExp.max(),
                              size=2),
                  selector=dict(type='histogram'))

#fig0.update_layout(yaxis_range=[0, 22])
fig0.update_layout(xaxis_range=[gpm.lifeExp.min(), gpm.lifeExp.max()])
fig0.show()

#### A more cumbersome alternative is below.  If we weren't using the selector parameter in our update_traces() call this is what we would have to do.

Here we manually set the nbinsx parameter of our Histogram trace to None and then iterate over our frames, first using the built-in dictionary update method to update the nbinsx parameter of each frame to None before specifying the xbins parameter details for each frame.

In [10]:
#fig0.data[0].nbinsx = None

#for i in fig0.frames:
#    i.data[0].update(nbinsx=None,
#                    xbins=dict(
#    start=gpm.lifeExp.min(),
#    end=gpm.lifeExp.max(),
#    size=2))

#fig0.show()

#### Note that the start and end parameteres in the xbins dict are optional.  Let's update with just size based on the sqrt methodology.

In [11]:
bin_size = (gpm.lifeExp.max() - gpm.lifeExp.min()) / np.sqrt(len(gpm))

fig0.update_traces(xbins=dict(size=bin_size),      
                  selector=dict(type='histogram'))

fig0.update_layout(yaxis_range=[0, 16])
# alternatively, fig0.update_traces(xbins_size=bin_size) ....using magic undersores.


"""Again, we could accomplish the same thing by operating on the frames directly.

for i in fig0.frames:
    i.data[0].update(xbins=dict(size=bin_size))

fig0.update_layout(yaxis_range=[0, 16])
"""

fig0.show()

#### Details on the magic underscore notation can be found at the following link.  Essentially, this is a syntactical shortcut that allows us to more easily set nested parameters within the figure tree.  See the initial plotting code for our life exp histogram.  The following arguments were passed to update_layout()...

font_family='Times New Roman',

title_font_family='Times New Roman',

title_font_size=22

#### [Magic underscore notation](https://plotly.com/python/creating-and-updating-figures/?_gl=1*u5kmto*_ga*MTkzOTU1ODQ3NC4xNjc3MTczMzIx*_ga_6G7EE0JNSC*MTY3NzYyNTI0NC40LjEuMTY3NzYyNjQwNS4wLjAuMA..#magic-underscore-notation)


#### Look at the figure tree to see how these values are stored.

In [12]:
fig0.layout

Layout({
    'barmode': 'relative',
    'font': {'family': 'Times New Roman'},
    'legend': {'tracegroupgap': 0},
    'sliders': [{'active': 0,
                 'currentvalue': {'prefix': 'year='},
                 'len': 0.9,
                 'pad': {'b': 10, 't': 60},
                 'steps': [{'args': [['1952'], {'frame': {'duration': 0, 'redraw':
                                     True}, 'mode': 'immediate', 'fromcurrent':
                                     True, 'transition': {'duration': 0, 'easing':
                                     'linear'}}],
                            'label': '1952',
                            'method': 'animate'},
                           {'args': [['1957'], {'frame': {'duration': 0, 'redraw':
                                     True}, 'mode': 'immediate', 'fromcurrent':
                                     True, 'transition': {'duration': 0, 'easing':
                                     'linear'}}],
                            'label': '195

#### Without magic underscores we'd have to specify values with nested dictionaries....this can get cumbersome.

In [13]:
fig0.update_layout(font = dict(family = 'Courier New'),
                  title = dict(font=dict(family = 'Courier New',
                                        size = 14)))

fig0.show()

#### We'll set everything back to what it was using magic underscores.

In [14]:
fig0.update_layout(font_family='Times New Roman',
                   title_font_family='Times New Roman',
                   title_font_size=22)

fig0.show()

#### Let's change the histnorm parm of fig0.  Instead of the default, 'count', values, we'll use probability.  We'll need to update the yaxis title accordingly.  Alternatively, this parameter can be specified through the initial px funciton call.  

#### You'll still see the 'name' count used in the tooltip....we'll change that as well.

In [15]:
fig0.update_traces(histnorm='probability',
                  selector=dict(type='histogram'))

fig0.update_layout(yaxis_title_text='probability')

for f in fig0.frames:
    temp = f.data[0].hovertemplate.replace('count', 'probability')
    f.data[0].update(hovertemplate=temp)

# fig0.update_layout(yaxis_range=None) 

fig0.show()

#### We could also show the cumulative distribution function as follows.  We'll set histnorm to 'probability' and set cumulative to True.  Let's reinitialize the whole plot.

In [16]:
fig1 = px.histogram(gpm, 'lifeExp',
                    animation_frame='year',
                    histnorm='probability',
                    cumulative=True,
                    color_discrete_sequence = ['black'] * 12,
                    range_x=[np.floor(gpm.lifeExp.min()),
                            np.ceil(gpm.lifeExp.max())],
                    title='Global Life Expectancy Distribution 1952 - 2007')

fig1.update_traces(xbins_size=bin_size) # using the sqrt method derived bin_size value

fig1.update_layout(font_family='Times New Roman',
                   title_font_family='Times New Roman',
                   title_font_size=22)

fig1.update_traces(marker_line=dict(width=1, color='red'))

fig1.show()

#### Similar to Seaborn, px has a dedicated function for plotting empirical cumulative distributions. 

In [17]:
fig2 = px.ecdf(gpm, 'lifeExp',
              animation_frame='year',
              color_discrete_sequence=['red'] * 12,
              range_x=[np.floor(gpm.lifeExp.min()),
                            np.ceil(gpm.lifeExp.max())])
fig2.show()

#### Let's see if we can add the ecdf over the hist....we might be tempted to do the following, but as we'll see, we're using the same y axis which will constrict our secondary plot considerably. 

In [18]:
fig3 = px.histogram(gpm, 'lifeExp',
                    animation_frame='year',
                    color_discrete_sequence = ['black'] * 12,
                    title='Global Life Expectancy Distribution 1952 - 2007')

fig3.update_traces(xbins_size=bin_size)

fig3.update_layout(font_family='Times New Roman',
                   title_font_family='Times New Roman',
                   title_font_size=22)

fig3.update_traces(marker_line=dict(width=1, color='red'))

fig3.add_trace(fig2.data[0])

#### Let's try this instead.  We'll need to use make_subplots().  Here we'll just produce a single-unanimated plot.

In [19]:
# Reinitializing fig2
fig2 = px.ecdf(gpm, 'lifeExp',
              color_discrete_sequence=['red'] * 12)

# Reinitializing fig3
fig3 = px.histogram(gpm, 'lifeExp',
                    histnorm='probability density',
                    color_discrete_sequence = ['black'])

fig3.update_traces(xbins_size=bin_size) # using the sqrt bins size established above

# Create subplots
figFinal = make_subplots(specs=[[{'secondary_y': True}]]) # takes a list of lists of dict

# Add relevant traces to figure
figFinal.add_trace(fig3.data[0], secondary_y=False) # add hist
figFinal.add_trace(fig2.data[0], secondary_y=True) # add ecdf

# Add figure title
figFinal.update_layout(title_text='Global Life Expectancy: CDF over PDF')

figFinal.show()

#### The figure factory submodule offers some nice functionality for displots.  The ff.create_distplot() function will return a histogram, kde plot and marginal rug plot with very little code.

#### Note: Figure factory was created prior to express, and now many of the figure factory functions are being deprectaed as new functionality is added to express. 

In [20]:
gpm_2007 = gpm.query('year==2007')

fig5 = ff.create_distplot([gpm_2007['lifeExp']], ['lifeExp'])

fig5.show()

#### Graph Objects Figure Creation

Plotly recommends using Express for most plots, but there are situations where GO is preferred.  Some plots, specifically some 3D traces, aren't implemented in Express.  Subplotting with different types of express plots is also tricky.  The recommendation is to start with an empty figure or subplot figure and to add traces.  Documentation pages for types of plots first list the details for creating such a plot in Express if functionality is available.

In [21]:
fig6 = go.Figure()

In [22]:
# A simple GO scatter plot.  Notice the tooltip.
fig6.add_traces(
    go.Scatter(x=gpm['gdpPercap'], y=gpm['lifeExp'], 
               mode='markers')
)

fig6.update_xaxes(title_text='GDP Per Cap')
fig6.update_yaxes(title_text='Life Expectancy')

fig6.show()

#### Notice the sparsity of the figure tree for the figure built with go.  We have to specify everything.

In [23]:
print(fig6)

Figure({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'x': array([779.4453145, 820.8530296, 853.10071  , ..., 792.4499603, 672.0386227,
                          469.7092981]),
              'y': array([28.801, 30.332, 31.997, ..., 46.809, 39.989, 43.487])}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'GDP Per Cap'}},
               'yaxis': {'title': {'text': 'Life Expectancy'}}}
})


#### We can specify data (a list of traces) and layout in our initial call to go.Figure() as well. Here we'll add an OLS regression line.  Notice we're passing a list of traces to the data param of go.Figure().

In [24]:
import statsmodels.api as sm

X = gpm['gdpPercap'].tolist()
y = gpm['lifeExp'].tolist()
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
preds = model.predict(X)

pio.templates.default = "plotly_dark"


fig6b = go.Figure(data=[
    go.Scatter(
        x=gpm['gdpPercap'], 
        y=gpm['lifeExp'], 
        mode='markers', 
        name='scatter'), 
    go.Scatter(
        x=X[:,1], 
        y=preds, 
        mode='lines', 
        name='reg')],
                  layout=go.Layout(
                      title=go.layout.Title(text='Per Cap GDP vs Life Expectancy')
                  )
)

fig6b.update_xaxes(title_text='GDP Per Cap')
fig6b.update_yaxes(title_text='Life Expectancy')

fig6b.show()

#### The same figure, but build with Express.  Note that we can just utilize the trendline parameter here instead of computing the OLS regression predictions manually.  We also get a better tooltip with no additional effort.

In [25]:
fig6c = px.scatter(gpm, 
                   x='gdpPercap', 
                   y='lifeExp', 
                   trendline='ols',
                   labels={'gdpPercap': 'GDP Per Cap',
                         'lifeExp': 'Life Expectancy'},
                   title='Per Cap GDP vs Life Expectancy')

fig6c.update_layout(title_y=0.95, title_x=0.05)

fig6c.data[1].marker.color = 'red'

fig6c.show()

#### Remember, go.Figure() takes a list of traces, and figures created with Express have trace data associated with them. We can build these traces with the go API and pass them or add them to our figures, but alternatively, we can use the select_traces() method to extract traces from an existing figure (including one built with Express).

In [26]:
# note that select_traces() returns a generator.
# As expected, our px version of the scatter with trend consists of two traces.

t1 = list(fig6c.select_traces())
print(t1)

[Scattergl({
    'hovertemplate': 'GDP Per Cap=%{x}<br>Life Expectancy=%{y}<extra></extra>',
    'legendgroup': '',
    'marker': {'color': '#636efa', 'symbol': 'circle'},
    'mode': 'markers',
    'name': '',
    'showlegend': False,
    'x': array([779.4453145, 820.8530296, 853.10071  , ..., 792.4499603, 672.0386227,
                469.7092981]),
    'xaxis': 'x',
    'y': array([28.801, 30.332, 31.997, ..., 46.809, 39.989, 43.487]),
    'yaxis': 'y'
}), Scattergl({
    'hovertemplate': ('<b>OLS trendline</b><br>lifeEx' ... ' <b>(trend)</b><extra></extra>'),
    'legendgroup': '',
    'marker': {'color': 'red', 'symbol': 'circle'},
    'mode': 'lines',
    'name': '',
    'showlegend': False,
    'x': array([   241.1658765,    277.5518587,    298.8462121, ..., 108382.3529   ,
                109347.867    , 113523.1329   ]),
    'xaxis': 'x',
    'y': array([ 54.14002447,  54.16785548,  54.18414316, ..., 136.85534205,
                137.59384703, 140.78743547]),
    'yaxis': 'y'
}

#### Now we can just pass this list of traces to go.Figure()

In [27]:
# Note that we didn't caputre the layout information from our PX plot,
# but we can customize using update_layout()

fig6d = go.Figure(data=t1)
fig6d.show()

#### Some adjustments to this new figure created by extracting the traces from our express plot.

In [28]:
fig6d['data'][0]['marker']['color'] = 'aqua'    

fig6d.update_xaxes(title_text='GDP Per Cap')
fig6d.update_yaxes(title_text='Life Expectancy')

fig6d.update_layout(title_text='Per Cap GDP vs Life Expectancy',
                   xaxis_range=[0,55000]) # restricting the range of the x axis here

fig6d.show()

#### And here we compare an MLP regressor with simple OLS regression.

In [29]:
from sklearn.neural_network import MLPRegressor

X2 = np.array([gpm['gdpPercap']]).T
y = gpm['lifeExp']

model = MLPRegressor(activation='tanh', solver='lbfgs')
model.fit(X2, y)

mlp_preds = model.predict(np.array(gpm['gdpPercap'])[:,None])

gpm['preds'] = mlp_preds
gpm.sort_values(by='preds', ascending=True, inplace=True)

/mnt/c/Users/rino2/OneDrive/Documentos/Maestria/Fourth Quarter - Summer 2023/Data Science Tools 1/DataScienceTools1/dst1_env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



In [30]:
fig6d.add_trace(go.Scatter(
    x=gpm['gdpPercap'], 
    y=gpm['preds'], 
    mode='lines', 
    name='MLP',
    yaxis='y2'))

#fig6d.data[2]['marker']['color'] = '#f56342'

#scatter, l1, l2 = fig6d.data

#fig6d.data = scatter, l1, l2

fig6d.update_layout(yaxis=dict(visible=False),
                    yaxis2=dict(overlaying='y'))

fig6d.update_layout(yaxis=dict(range=[gpm['lifeExp'].min(), gpm['lifeExp'].max()]))
fig6d.update_layout(yaxis2=dict(range=[gpm['lifeExp'].min(), gpm['lifeExp'].max()]))

fig6d.show()

#### Order of traces in the data node should control rendering order, with traces being rendered based on order of appearance.

Typcially you can unpack these and reorder them: 

__l1, l2, scatter = fig.data__

__fig.data = scatter, l1, l2__

#### This approach might fail when integrating traces generated by go and px into a single figure.  Below we plot our scatter and model predictions using only go functions.

In [31]:
fig6e = go.Figure(data=[
    go.Scatter(
        x=gpm['gdpPercap'], 
        y=gpm['lifeExp'], 
        mode='markers',
        marker=dict(color='rgba(3, 252, 211, 0.25)'), # defining opacity with CSS rgba (red, green, blue, alpha)
        name='scatter'), 
    go.Scatter(
        x=X[:,1], 
        y=preds, 
        mode='lines', 
        name='reg'),
    go.Scatter(
        x=gpm['gdpPercap'], 
        y=gpm['preds'], 
        mode='lines', 
        name='MLP',
        line_color='#6f03fc'
    )],
                  layout=go.Layout(
                      title=go.layout.Title(text='Per Cap GDP vs Life Expectancy')
                  )
)

fig6e.update_xaxes(title_text='GDP Per Cap')
fig6e.update_yaxes(title_text='Life Expectancy')

fig6e.update_layout(xaxis_range=[0,55000])
fig6e.update_layout(yaxis_range=[20, 90])

fig6e.show()

#### Tables in Plotly

We'll look at tables using GO here.  Dash also has some specific table creation functionality that we'll look at next week.

In [32]:
# Here we create a table based on dictionary representations of our headers and cells.
# We'll use the go.Table function and pass the resulting trace to go.Figure.

pio.templates.default = 'simple_white'

header = {'values': ['feat_1', 'feat_2']
         }

cells = {'values': [[1, 2, 3, 4, 5],
                    ['foo', 'bar', 'norf', 'spam', 'eggs']]
        }

fig7 = go.Figure(data=go.Table(header=header,
                              cells=cells))

fig7.show()

#### The header and cells parameters from go.Figure take a dictionary which supports data and style arguments.

In [33]:
header = dict(values=['feat_1', 'feat_2'],
              line_color='black',
              fill_color='lightgray',
              align='center')

cells = dict(values= [[1, 2, 3, 4, 5],
                    ['foo', 'bar', 'norf', 'spam', 'eggs']],
             line_color='black',
             fill_color='#d8f2f1',
             align='left')

fig7 = go.Figure(data=[go.Table(
    header=header,
    cells=cells)])

fig7.update_traces(cells_font_size=10)

fig7.show()

#### Using a Pandas DataFrame

In [34]:
chars = pd.read_csv('https://nces.ed.gov/ipeds/datacenter/data/HD2022.zip', 
                    compression='zip',
                    encoding="ISO-8859-1")

In [35]:
chars = chars.loc[chars['STABBR'] == 'CO', :]

In [36]:
fig8 = go.Figure(data=[go.Table(
    header=dict(values=['INSTNM', 'CITY', 'COUNTYNM','CHFNM'],
                line_color='black',
                fill_color='#d8f2f1',
                align='center'),
    cells=dict(values=[chars.INSTNM, chars.CITY, chars.COUNTYNM, chars.CHFNM],
               line_color='black',
               fill_color='white',
               align='left'),
    columnwidth=[2.5, 1, 1, 1.5])
])

fig8.update_traces(cells_font_size=10)

fig8.show()

#### Dash

Anytime you use fig.show() you can render that same figure as a Dash application.  All you need to do is pass the figure object to the figure argument of the dash_core_components Graph component.

In [37]:
app = Dash()

app.layout = html.Div([
    dcc.Graph(figure=fig8)
])

app.run_server(debug=True, use_reloader=False)

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/mnt/c/Users/rino2/OneDrive/Documentos/Maestria/Fourth Quarter - Summer 2023/Data Science Tools 1/DataScienceTools1/dst1_env/lib/python3.10/site-packages/werkzeug/serving.py", line 746, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/mnt/c/Users/rino2/OneDrive/Documentos/Maestria/Fourth Quarter - Summer 2023/Data Science Tools 1/DataScienceTools1/dst1_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_376/2740889978.py", line 7, in <module>
    app.run_server(debu

TypeError: object of type 'NoneType' has no len()

#### Let's read in some more data for plotting.

In [ ]:
enr = pd.read_csv('https://nces.ed.gov/ipeds/datacenter/data/EFFY2022.zip',
                  compression='zip',
                  encoding="ISO-8859-1")

enr = enr.loc[enr['EFFYALEV'] == 1, ['UNITID', 'EFYTOTLT']]

data = pd.merge(chars, enr, how='left', on='UNITID')

data = pd.merge(chars, enr, how='left', on='UNITID')

data.dropna(how='any', axis=0, inplace=True)

data.rename(columns={'INSTNM': 'school',
                    'STABBR': 'state',
                    'CONTROL': 'control',
                    'EFYTOTLT': 'enrollment'},
           inplace=True)

In [ ]:
data.head()

#### Let's look at something a bit more complex.  Here we introduce the callback.

In [ ]:
app = Dash()

md_text = """
# CO Higher Education Institutions
## Use the dropdown to view data for a specific county."""

app.layout = html.Div([
    html.Div([
        dcc.Markdown(children=md_text, 
                     style={'font': '15px Arial, sans-serif', 'color': 'black', 'font-weight': '500'}),
        dcc.Dropdown(options=[dict(label=i, value=i) for i in data.COUNTYNM.unique()],
                    value='Denver County',
                    id='county-selection'
        )
    ], style={'width': '28%', 'padding': '15px', 'display': 'inline-block', 'backgroundColor': 'white'}),
    
    html.Div([
        html.Div([
            dcc.Graph(id='enr-strip'),
        ], style={'width': '48%', 'display': 'inline-block', 'backgroundColor': 'white'}),
        html.Div([
            dcc.Graph(id='enr-table'),
            ], style={'width': '48%','float': 'right', 'display': 'inline-block', 'backgroundColor': 'black'})
        
    ])
], style={'backgroundColor': 'white'})

@app.callback(
    Output(component_id='enr-strip', component_property='figure'),
    Output(component_id='enr-table', component_property='figure'),
    Input(component_id='county-selection', component_property='value'))
def update_plots(cnty):
    pio.templates.default='simple_white'
    
    figa = px.strip(data.loc[data['COUNTYNM']==cnty, :], 'enrollment',
                        color_discrete_sequence = ['black'] * 12,
                        title='Total Enrollment Distribution',
                        hover_data={'CITY': True,
                                    'school': True})

    figa.update_layout(font_family='Times New Roman',
                   title_font_family='Times New Roman',
                   title_font_size=22)
    
    figb = go.Figure(data=[go.Table(
        header=dict(values=['INSTNM', 'CITY', 'COUNTYNM','CHFNM'],
                    line_color='black',
                    fill_color='#d8f2f1',
                    align='center'),
        cells=dict(values=[chars.loc[chars['COUNTYNM']==cnty,:].INSTNM, 
                           chars.loc[chars['COUNTYNM']==cnty,:].CITY, 
                           chars.loc[chars['COUNTYNM']==cnty,:].COUNTYNM, 
                           chars.loc[chars['COUNTYNM']==cnty,:].CHFNM],
                   line_color='black',
                   fill_color='white',
                   align='left'),
        columnwidth=[2.5, 1, 1, 1.5])
        ])

    figb.update_traces(cells_font_size=10)
    
    return figa, figb

app.run_server(debug=True, use_reloader=False)

In [ ]:
app = Dash()

md_text = """
# CO Higher Education Institution Enrollment by County
## Use the dropdown to view data for a specific county.
### _Institutions may be excluded from the plot by selecting from the legend._"""

app.layout = html.Div([
    html.Div([
        dcc.Markdown(children=md_text, 
                     style={'font': '15px Arial, sans-serif', 'color': 'black', 'font-weight': '500'}),
        dcc.Dropdown(options=[dict(label=i, value=i) for i in sorted(data.COUNTYNM.unique())],
                    value='Denver County',
                    id='county-selection'),
        html.P('''Lorem ipsum dolor sit amet, consectetur adipiscing elit, 
               sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
               Ut enim ad minim veniam, quis nostrud exercitation ullamco 
               laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure 
               dolor in reprehenderit in voluptate velit esse cillum dolore eu 
               fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, 
               sunt in culpa qui officia deserunt mollit anim id est laborum.''')
    ], style={'width': '40%', 'padding': '15px', 'display': 'inline-block', 'backgroundColor': 'white'}),
    
    html.Div([dcc.Graph(id='enr-donut')],
              style={'width': '58%','float': 'right', 'display': 'inline-block', 'backgroundColor': 'black'})
], style={'backgroundColor': 'white'})

@app.callback(
    Output(component_id='enr-donut', component_property='figure'),
    Input(component_id='county-selection', component_property='value'))
def update_plots(cnty):
    pio.templates.default='simple_white'
    
    fig = px.pie(data.loc[data['COUNTYNM'] == cnty, :], 
             values='enrollment', 
             names='school',
            hover_data=['school'],
             color_discrete_sequence=px.colors.sequential.Inferno,
            hole=0.4)

    fig.update_traces(textposition='inside')
    
    return fig

app.run_server(debug=True, use_reloader=False)

#### In Class: Using the above examples build a simple dash application using the gapminder data.  Use a dcc.Dropwdown component to allow for year selection and render the global life expectancy distribution for that year.  You can borrow the histogram code from earlier in the notebook.  If you need an extra challenge, make an additional dcc object to control some other aspect of the resulting plot.